In [19]:
import sys
import os
import pandas as pd
import numpy as np
from Bio import SeqIO
import re

In [20]:
dataset = 'CF2017'

print('Loading raw data for', dataset, '...')
data = pd.read_csv('/Users/maryamkoddus/Documents/maryam-ko-QMUL-MSc-Project/01_input_data/raw_data/mmc4.csv', header=0)
print('Raw data loaded.')
data

Loading raw data for CF2017 ...
Raw data loaded.


,id,Uniprot,Gene names,Protein names,Sequence Window,Modified sequence,Aa Position,Known site,EOC 1,EOC 2,...,p value,adj p value,EOC vs FTE p value,EOC vs OSE_ p value,FTE vs OSE p value,PCA component 1,"Significant in Lawrence et al , 2014",Cluster,Protein identified,phopsphorylated site/protein significance
0,78,O00161,SNAP23,Synaptosomal-associated protein 23 (SNAP-23) (...,HQITDESLESTRR,_AHQITDES(ph)LESTRR_,S20,+,20.76,23.42,...,0.001520,0.012800,0.000277,0.066800,0.0506,NaN,NaN,A,YES,NaN
1,96,O00264,PGRMC1,Membrane-associated progesterone receptor comp...,PAASGDSDDDEPP,_GDQPAASGDS(ph)DDDEPPPLPR_,S57,+,28.07,27.17,...,0.000008,0.000438,0.000010,0.000009,0.6900,NaN,NaN,A,YES,+
2,158,O00767,SCD,Acyl-CoA desaturase (EC 1.14.19.1) (Delta(9)-d...,AVKEKGSTLDLSD,_EKGS(ph)TLDLSDLEAEK_,S198,+,19.39,NaN,...,0.000013,0.000608,0.000007,0.000048,0.0735,NaN,NaN,A,YES,NaN
3,182,O14578,CIT,Citron Rho-interacting kinase (CRIK) (EC 2.7.1...,RLHRRVSEVEAVL,_RVS(ph)EVEAVLSQK_,S108,+,NaN,NaN,...,0.003560,0.021800,0.001420,0.014600,0.1140,NaN,NaN,A,YES,NaN
4,199,O14647,CHD2,Chromodomain-helicase-DNA-binding protein 2 (C...,AVSDPRSPPSQKS,_(gl)QPLPPLHPAVSDPRS(ph)PPSQK_,S1789,NaN,21.77,22.94,...,0.002910,0.019400,0.004140,0.001140,0.8340,NaN,NaN,A,YES,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7652,11074,Q13523,PRPF4B,Serine/threonine-protein kinase PRP4 homolog (...,DNDITPYLVSRFY,_LCDFGSASHVADNDITPY(ph)LVSR_,Y849,+,26.83,23.45,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7653,11075,Q13627,DYRK1A,Dual specificity tyrosine-phosphorylation-regu...,GQRIYQYIQSRFY,_IYQY(ph)IQSR_,Y321,+,27.25,27.38,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7654,11077,Q7Z3U7,MON2,Protein MON2 homolog (Protein SF21),QDVLHRYIEDERL,_Y(ph)IEDER_,Y1622,NaN,25.32,27.13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7655,11080,Q8NBJ4,GOLM1,Golgi membrane protein 1 (Golgi membrane prote...,LRGEDDYNMDENE,_LRGEDDY(ph)NM(ox)DENEAESETDK_,Y341,NaN,NaN,26.14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
data.columns = data.columns.str.strip()

In [22]:
# Filter out rows that contain '/' or ';' in the 'Aa Position' or 'Gene names' columns
data = data[~data['Aa Position'].str.contains(r'[/;]', na=False)]  # Removes rows with '/' or ';' in 'Aa Position'
data = data[~data['Gene names'].str.contains(r'[/;]', na=False)]  # Removes rows with '/' or ';' in 'Gene names'


In [23]:
# filter data
data['Sequence Window'] = data['Sequence Window'].str.replace('_', '')

In [24]:
def match_seq_to_genename(dataset, seq_column):
    '''
    Maps amino acid sequences to gene names using the loaded fasta file.
    
    args:
    =====
    dataset: <pd.Dataframe> with a column of amino acid sequences
    seq_column: <str> column name containing amino acid sequences
    
    out:
    ====
    dataset: <pd.Dataframe> with an additional column containing gene names
    '''    
    
    fasta_sequence = list(SeqIO.parse(f"/Users/maryamkoddus/Documents/maryam-ko-QMUL-MSc-Project/01_input_data/raw_data/UP000005640_9606.fasta", "fasta"))
    
    gene_dict = {}
    
    # iterate over rows in seq_column
    for i in dataset[seq_column]:
        print(i)
        i_str = str(i)
        for seq_record in fasta_sequence:
            matches = re.findall(i_str, str(seq_record.seq))
            if matches:
                print(f"Match found for sequence: {seq_record}")
                gene_name_match = re.search(r"GN=(\w+)", seq_record.description)
                print('Gene name match:', gene_name_match)
                if gene_name_match:
                    gene_name = gene_name_match.group(1)
                    gene_dict[i] = gene_name
                    print(f"Match found: {i_str} -> {gene_name}")
                else:
                    print(f"No gene name match found in description for sequence: {i_str}")

    # map sequences to gene names           
    dataset['GeneName'] = dataset[seq_column].map(gene_dict) 
    print('Amino acid sequences matched to gene names.')

    return dataset

In [ ]:
data = match_seq_to_genename(data, 'Sequence Window')

HQITDESLESTRR
Match found for sequence: ID: sp|O00161|SNP23_HUMAN
Name: sp|O00161|SNP23_HUMAN
Description: sp|O00161|SNP23_HUMAN Synaptosomal-associated protein 23 OS=Homo sapiens OX=9606 GN=SNAP23 PE=1 SV=1
Number of features: 0
Seq('MDNLSSEEIQQRAHQITDESLESTRRILGLAIESQDAGIKTITMLDEQKEQLNR...IDS')
Gene name match: <re.Match object; span=(81, 90), match='GN=SNAP23'>
Match found: HQITDESLESTRR -> SNAP23
PAASGDSDDDEPP
Match found for sequence: ID: sp|O00264|PGRC1_HUMAN
Name: sp|O00264|PGRC1_HUMAN
Description: sp|O00264|PGRC1_HUMAN Membrane-associated progesterone receptor component 1 OS=Homo sapiens OX=9606 GN=PGRMC1 PE=1 SV=3
Number of features: 0
Seq('MAAEDVVATGADPSDLESGGLLHEIFTSPLNLLLLGLCIFLLYKIVRGDQPAAS...KND')
Gene name match: <re.Match object; span=(100, 109), match='GN=PGRMC1'>
Match found: PAASGDSDDDEPP -> PGRMC1
AVKEKGSTLDLSD
Match found for sequence: ID: sp|O00767|SCD_HUMAN
Name: sp|O00767|SCD_HUMAN
Description: sp|O00767|SCD_HUMAN Stearoyl-CoA desaturase OS=Homo sapiens OX=9606 

In [17]:
data

,id,Uniprot,Gene names,Protein names,Sequence Window,Modified sequence,Aa Position,Known site,EOC 1,EOC 2,...,adj p value,EOC vs FTE p value,EOC vs OSE_ p value,FTE vs OSE p value,PCA component 1,"Significant in Lawrence et al , 2014",Cluster,Protein identified,phopsphorylated site/protein significance,GeneName
0,78,O00161,SNAP23,Synaptosomal-associated protein 23 (SNAP-23) (...,HQITDESLESTRR,_AHQITDES(ph)LESTRR_,S20,+,20.76,23.42,...,0.012800,0.000277,0.066800,0.0506,NaN,NaN,A,YES,NaN,SNAP23
1,96,O00264,PGRMC1,Membrane-associated progesterone receptor comp...,PAASGDSDDDEPP,_GDQPAASGDS(ph)DDDEPPPLPR_,S57,+,28.07,27.17,...,0.000438,0.000010,0.000009,0.6900,NaN,NaN,A,YES,+,PGRMC1
2,158,O00767,SCD,Acyl-CoA desaturase (EC 1.14.19.1) (Delta(9)-d...,AVKEKGSTLDLSD,_EKGS(ph)TLDLSDLEAEK_,S198,+,19.39,NaN,...,0.000608,0.000007,0.000048,0.0735,NaN,NaN,A,YES,NaN,SCD
3,182,O14578,CIT,Citron Rho-interacting kinase (CRIK) (EC 2.7.1...,RLHRRVSEVEAVL,_RVS(ph)EVEAVLSQK_,S108,+,NaN,NaN,...,0.021800,0.001420,0.014600,0.1140,NaN,NaN,A,YES,NaN,CIT
4,199,O14647,CHD2,Chromodomain-helicase-DNA-binding protein 2 (C...,AVSDPRSPPSQKS,_(gl)QPLPPLHPAVSDPRS(ph)PPSQK_,S1789,NaN,21.77,22.94,...,0.019400,0.004140,0.001140,0.8340,NaN,NaN,A,YES,NaN,CHD2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7652,11074,Q13523,PRPF4B,Serine/threonine-protein kinase PRP4 homolog (...,DNDITPYLVSRFY,_LCDFGSASHVADNDITPY(ph)LVSR_,Y849,+,26.83,23.45,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PRP4K
7653,11075,Q13627,DYRK1A,Dual specificity tyrosine-phosphorylation-regu...,GQRIYQYIQSRFY,_IYQY(ph)IQSR_,Y321,+,27.25,27.38,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DYRK1B
7654,11077,Q7Z3U7,MON2,Protein MON2 homolog (Protein SF21),QDVLHRYIEDERL,_Y(ph)IEDER_,Y1622,NaN,25.32,27.13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MON2
7655,11080,Q8NBJ4,GOLM1,Golgi membrane protein 1 (Golgi membrane prote...,LRGEDDYNMDENE,_LRGEDDY(ph)NM(ox)DENEAESETDK_,Y341,NaN,NaN,26.14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GOLM1


In [ ]:
# Extract amino acid (first character) and position (rest of the string)
data['Amino_Acid'] = data['Aa Position'].str[0]  # Extract the first character (amino acid)
data['Position'] = data['Aa Position'].str[1:]  # Extract everything after the first character (position)


In [ ]:
data['Phosphosite'] = data['Amino_Acid'].astype(str) + '(' + data['Position'].astype(str) + ')'
print(data[['Aa Position', 'Amino_Acid', 'Position', 'Phosphosite']].head())

In [ ]:
# Keep only 'Phosphosite' and ratio columns
keepcols = ['Phosphosite', 'GeneName'] + [
    col for col in data.columns if 'EOC' in col or 'FTE' in col or 'OSE' in col or 'AvgExp' in col
]
data = data[keepcols]
data

KeyError: "['Phosphosite'] not in index"

In [ ]:
print("Data after subsetting columns:", data)

In [ ]:
print("Cols after subsetting:", data.columns)

In [ ]:
# Define the list of raw expression columns that need to be log2 transformed
raw_columns = [
    'EOC 1', 'EOC 2', 'EOC 3', 'EOC 4',
    'FTE 13', 'FTE 14', 'FTE 15', 'FTE 16', 'FTE 17',
    'OSE 26', 'OSE 27', 'OSE 28', 'OSE 29',
    'AvgExp'
]

data[raw_columns] = data[raw_columns].apply(pd.to_numeric, errors="coerce")
data[raw_columns] = data[raw_columns].replace([np.inf, -np.inf], np.nan)

In [ ]:
def log2_transform(dataset):
    '''
    Log2 transform a dataset.
    
    args:
    =====
    dataset: <pd.Dataframe>
    
    out:
    ====
    dataset: <pd.Dataframe> with log2 transformed values

    '''
    cols_to_transform = dataset[raw_columns]
    dataset[raw_columns] = cols_to_transform.apply(np.log2)
    print('Data has been log2 transformed.')
    return dataset

In [ ]:
# Apply the log2 transformation
data = log2_transform(data)
print(f"DataFrame after log2 transformation:\n{data}")

In [ ]:
def create_phos_ID(dataset):
    '''
    Concatenates GeneName and Phosphosite columns.
    
    args:
    =====
    dataset: <pd.Dataframe> with columns 'GeneName' and 'Phosphosite'
    
    out:
    ====
    dataset: <pd.Dataframe> with 'phosphosite_ID' column and 'GeneName' + 'Phosphosite' columns dropped
    '''
    dataset.loc[:, 'phosphosite_ID'] = dataset['GeneName'].astype(str) + '_' + dataset['Phosphosite'].astype(str)
    dataset = dataset.drop(columns=['Phosphosite', 'GeneName'])
    print('Phosphosite IDs created.')
    return dataset

data = create_phos_ID(data) # call function to create phosphosite_ID column

print('Phosphosite IDs created.')

In [ ]:
def clean_phosID_col(data):
    data = data[~data.phosphosite_ID.str.contains('nan', case = False)]
    data = data[~data.phosphosite_ID.str.contains(';', case = False)] # remove rows containing ';' in phosphosite_ID column
    data = data[~data.phosphosite_ID.str.contains('-', case = False)] # remove rows containing '-' in phosphosite_ID column
    
    # check whether there are any phosphosites with multiple measurements
    data_grouped = data.groupby(by = 'phosphosite_ID')
    if len(data) != len(data_grouped):
        data = data_grouped.mean()
        data.reset_index(inplace=True) # reset index
        print('Phosphosites with multiple measurements have been averaged')
    else:
        print('There are no phosphosites with multiple measurements')
        
    print(data)
        
    data = data.replace([np.inf, -np.inf], np.nan)
        
    if data.columns[0] != 'phosphosite_ID':
        phosphosite_ID = data.pop('phosphosite_ID')
        data.insert(0, 'phosphosite_ID', phosphosite_ID)
    return data

In [ ]:
data = clean_phosID_col(data)
print("After cleaning phosphosite_ID column:")
data

In [ ]:
data.to_csv(f'/Users/maryamkoddus/Documents/maryam-ko-QMUL-MSc-Project/01_input_data/PreprocessedDatasets/KS2014.csv', index=False)


print(dataset, 'has been saved to CSV successfully!', data)                       